# Set up

In [1]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Creating and running a Graph

In [2]:
import tensorflow as tf
import numpy as np
reset_graph()
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [3]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [5]:
sess.close()

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [7]:
print(result)

42


In [8]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [9]:
print(result)

42


In [10]:
sess.close()

# Managing graphs

In [11]:
reset_graph()
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [12]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [13]:
x2.graph is tf.get_default_graph()

False

In [14]:
w = tf.constant(3)
x = w + 3
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

11
18


In [15]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

11
18


# Linear Regression

## Normal Equation

In [16]:
import numpy as no
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
housing = fetch_california_housing()
m, n = housing.data.shape
scaled_data = StandardScaler().fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_data]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [17]:
theta_value

array([[ 2.06856298],
       [ 0.82961965],
       [ 0.11875178],
       [-0.26552707],
       [ 0.30569667],
       [-0.00450281],
       [-0.03932635],
       [-0.8998825 ],
       [-0.87053877]], dtype=float32)

## Manually Computing the Gradients

In [18]:
from sklearn.preprocessing import StandardScaler
n_epoch = 1000
learning_rate = 0.01
data = housing.data
m, n = data.shape
scaled_data = StandardScaler().fit_transform(data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_data]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradient = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradient)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print("Epoch", epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
best_theta

Epoch 0 MSE= 14.7726
Epoch 100 MSE= 0.964645
Epoch 200 MSE= 0.68937
Epoch 300 MSE= 0.640193
Epoch 400 MSE= 0.608236
Epoch 500 MSE= 0.585242
Epoch 600 MSE= 0.568597
Epoch 700 MSE= 0.556536
Epoch 800 MSE= 0.547789
Epoch 900 MSE= 0.541443


array([[ 2.06855249],
       [ 0.80584002],
       [ 0.15618883],
       [-0.14223813],
       [ 0.16928515],
       [ 0.00942415],
       [-0.041986  ],
       [-0.64930242],
       [-0.61339927]], dtype=float32)

### Using autodiff

In [19]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradient = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradient)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print("Epoch", epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
best_theta

Epoch 0 MSE= 10.5926
Epoch 100 MSE= 0.699455
Epoch 200 MSE= 0.571924
Epoch 300 MSE= 0.557352
Epoch 400 MSE= 0.548313
Epoch 500 MSE= 0.541799
Epoch 600 MSE= 0.537074
Epoch 700 MSE= 0.533641
Epoch 800 MSE= 0.531144
Epoch 900 MSE= 0.529327


array([[  2.06855249e+00],
       [  7.70412445e-01],
       [  1.29901633e-01],
       [ -1.12047002e-01],
       [  1.60478979e-01],
       [  3.21454077e-04],
       [ -3.90070044e-02],
       [ -8.74200284e-01],
       [ -8.35951090e-01]], dtype=float32)

### Using an Optimizer
\# 1

In [20]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print("Epoch", epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
best_theta

Epoch 0 MSE= 12.6593
Epoch 100 MSE= 0.997471
Epoch 200 MSE= 0.712683
Epoch 300 MSE= 0.656459
Epoch 400 MSE= 0.619779
Epoch 500 MSE= 0.593432
Epoch 600 MSE= 0.574393
Epoch 700 MSE= 0.560623
Epoch 800 MSE= 0.55066
Epoch 900 MSE= 0.543448


array([[ 2.06855226],
       [ 0.72979027],
       [ 0.14530627],
       [ 0.00776081],
       [ 0.0424718 ],
       [ 0.0064523 ],
       [-0.03944388],
       [-0.80004561],
       [-0.75498569]], dtype=float32)

\# 2

In [22]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print("Epoch", epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
best_theta

Epoch 0 MSE= 13.2244
Epoch 100 MSE= 0.532315
Epoch 200 MSE= 0.524476
Epoch 300 MSE= 0.524326
Epoch 400 MSE= 0.524321
Epoch 500 MSE= 0.524321
Epoch 600 MSE= 0.524321
Epoch 700 MSE= 0.524321
Epoch 800 MSE= 0.524321
Epoch 900 MSE= 0.52432


array([[ 2.06855798],
       [ 0.82962108],
       [ 0.11875191],
       [-0.26553035],
       [ 0.30569911],
       [-0.00450292],
       [-0.03932634],
       [-0.89988232],
       [-0.87053788]], dtype=float32)

# Feeding data to the training algorithm

## Palceholder nodes

In [23]:
reset_graph()
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})

In [24]:
B_val_1

array([[ 6.,  7.,  8.]], dtype=float32)

In [25]:
B_val_2

array([[  9.,  10.,  11.],
       [ 12.,  13.,  14.]], dtype=float32)

In [36]:
X = tf.placeholder(tf.float32, shape=(None, None), name='X')
y = tf.placeholder(tf.float32, shape=(None, None), name='y')
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch+batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target[indices].reshape(-1, 1)
    return X_batch, y_batch

theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        for batch_index  in range(n_batches):
            if epoch % 100 == 0:
                print("Epoch", epoch, 'MSE=', mse.eval())
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()
print(best_theta)

InvalidArgumentError: You must feed a value for placeholder tensor 'X_10' with dtype float
	 [[Node: X_10 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'X_10', defined at:
  File "/Users/gaufung/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/gaufung/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/gaufung/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/gaufung/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/gaufung/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/gaufung/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self

# Saving and Restoring Models

In [37]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print("Epoch", epoch, 'MSE=', mse.eval())
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model_final.ckpt')
best_theta

Epoch 0 MSE= 6.70569
Epoch 100 MSE= 0.530588
Epoch 200 MSE= 0.524822
Epoch 300 MSE= 0.524382
Epoch 400 MSE= 0.524329
Epoch 500 MSE= 0.524322
Epoch 600 MSE= 0.524321
Epoch 700 MSE= 0.524321
Epoch 800 MSE= 0.524321
Epoch 900 MSE= 0.524321


array([[ 2.06855774],
       [ 0.82962704],
       [ 0.11875308],
       [-0.26554146],
       [ 0.30570826],
       [-0.0045026 ],
       [-0.03932654],
       [-0.89986813],
       [-0.87052441]], dtype=float32)

In [41]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')

## Using TensorBoard

In [42]:
from datetime import datetime
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = "{}/run-{}".format(root_logdir, now)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print("Epoch", epoch, 'MSE=', mse.eval())
        sess.run(training_op)
        summary_str = mse_summary.eval()
        file_writer.add_summary(summary_str, epoch)
    best_theta = theta.eval()
best_theta

Epoch 0 MSE= 7.39793
Epoch 100 MSE= 0.536665
Epoch 200 MSE= 0.52478
Epoch 300 MSE= 0.524361
Epoch 400 MSE= 0.524326
Epoch 500 MSE= 0.524322
Epoch 600 MSE= 0.524321
Epoch 700 MSE= 0.524321
Epoch 800 MSE= 0.52432
Epoch 900 MSE= 0.524321


array([[ 2.06855774],
       [ 0.82961321],
       [ 0.11875053],
       [-0.26551551],
       [ 0.30568674],
       [-0.00450333],
       [-0.03932604],
       [-0.89989948],
       [-0.87055415]], dtype=float32)

# Name scopes

In [ ]:
reset_graph()
a1 = tf.Variable(0, name='a')
a2 = tf.Variable(0, name='a')
with tf.name_scope('param'):
    a3 = tf.Variable(0, name='a')
with tf.name_scope('param'):
    a4 = tf.Variable(0, name='a')
for node in (a1, a2, a3, a4):
    print(node.op.name)

# Modularity

In [ ]:
reset_graph()
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
w1 = tf.Variable(tf.random_normal((n_features, 1)), name='weights1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name='weights2')
b1 = tf.Variable(0.0, name='bias1')
b2 = tf.Variable(0.0, name='bias2')
z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')
relu1 = tf.maximum(z1, 0, name='relu1')
relu2 = tf.maximum(z2, 0, name='relu2')
ouput = tf.add(relu1, relu2, name='output')

In [ ]:
reset_graph()
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0., name='relu')

In [ ]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
ouput = tf.add_n(relus, name='output')

In [ ]:
file_writer = tf.summary.FileWriter('logs/relu1', tf.get_default_graph())

In [ ]:
file_writer.close()

# Sharing Variables

In [ ]:
reset_graph()
def relu(X, threshold):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, 0., name='max')

In [ ]:
threshold = tf.Variable(0.0, name='threshold')
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X, threshold) for _ in range(5)]
output = tf.add_n(relus, name='output')

In [ ]:
reset_graph()
with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold', shape=(),
                               initializer=tf.constant_initializer(0.0))

In [ ]:
with tf.variable_scope('relu', reuse=True):
    threshold =tf.get_variable('threshold')

In [ ]:
with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold = tf.get_variable('threshold')